In [8]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, field_validator
import os

In [2]:
load_dotenv()

True

In [3]:
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_ENDPOINT"] = ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = API_KEY

In [5]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    api_version="2023-03-15-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [13]:
class Router(BaseModel):
    endpoint: str = Field(
        description="The endpoint to route the request to"
    )

    @field_validator('endpoint')
    def validate_endpoint(cls, value):
        allowed_endpoints = ["check_smart", "fallback", "preferences_confirmation"]
        if value not in allowed_endpoints:
            raise ValueError(f"Invalid endpoint. Allowed endpoints are: {allowed_endpoints}")
        return value

In [14]:
def check_router(llm):
    structured_llm_smt_checker = llm.with_structured_output(Router)
    router_template = """
    You are tasked to route the discussion to one of the three endpoints:
    "check_smart": if the query is describing an investment goal explicitly!
    "preferences_confirmation": If the user is describing an investment preference(for instance i don't want to invest in it industry, in companies that support a war!)
    "fallback": If the investor is trying to chat or ask anything else
    """

    route_prompt = ChatPromptTemplate.from_messages(
        [("system", router_template), ("human", "The investor goal :\n\n {goal}")]
    )
    router = route_prompt | structured_llm_smt_checker
    return router

In [16]:
app = check_router(llm)
result = app.invoke({"goal": "I want to invest my money and get a lot of money!"})
print(result)

endpoint='check_smart'


In [20]:
type(result.endpoint)

str